# Programming with Python

## Lecture 13: Concurrency 5

### Armen Gabrielyan

#### Yerevan State University / ASDS

#### 17 May, 2025

# Async programming with Async IO

**Asynchronous programming** is a powerful paradigm for writing efficient, non-blocking code and Python's Async IO framework provides excellent tools for implementing it. Async IO is built around coroutines, event loops and the `async`/`await` syntax. The basic idea is to allow tasks to pause execution while waiting for slow operations (like network or file I/O), letting other tasks run during that wait time.

Async IO is a *single-threaded, single-process design* that uses **cooperative multitasking**. This is a fundamental characteristic that distinguishes it from other concurrency models:

1. **Single thread** - All code runs on a single thread, unlike threading or multiprocessing.   
2. **Explicit yield points** - Tasks voluntarily yield control at `await` statements, allowing other tasks to run.
3. **Non-preemptive** - The scheduler doesn't interrupt tasks; they must cooperate by yielding control.
4. **Event loop coordination** - The event loop manages the scheduling of coroutines as they yield control.

This cooperative approach means tasks must be well-behaved - a task that performs CPU-intensive operations without yielding will block the entire event loop, preventing other tasks from running.

### Async IO explained

Async IO may at first seem counterintuitive and paradoxical. How does something that facilitates concurrent code use a single thread and a single CPU core? Miguel Grinberg’s 2017 PyCon talk explains everything quite beautifully:

> Chess master Judit Polgár hosts a chess exhibition in which she plays multiple amateur players. She has two ways of conducting the exhibition: synchronously and asynchronously.
>
> Assumptions:
>
> - 24 opponents
> - Judit makes each chess move in 5 seconds
> - Opponents each take 55 seconds to make a move
> - Games average 30 pair-moves (60 moves total)
>
> **Synchronous version:** Judit plays one game at a time, never two at the same time, until the game is complete. > Each game takes (55 + 5) * 30 == 1800 seconds, or 30 minutes. The entire exhibition takes 24 * 30 == 720 minutes, or *12 hours*.
> 
> **Asynchronous version:** Judit moves from table to table, making one move at each table. She leaves the table and lets the opponent make their next move during the wait time. One move on all 24 games takes Judit 24 * 5 == 120 seconds, or 2 minutes. The entire exhibition is now cut down to 120 * 30 == 3600 seconds, or just *1 hour*. 
>
> [Source](https://www.youtube.com/watch?v=iG6fr81xHKA&t=269s)

## Key components:

1. **(Native) Coroutines**: Functions defined with `async def` that can be paused at `await` points to let other coroutines run and later be resumed.
2. **Event loop**: The central execution mechanism that runs and manages coroutines.
3. **Tasks**: Wrapped coroutines that run concurrently in the event loop.
4. **Awaitables**: Objects that can be used with the `await` keyword (coroutines, Tasks, Futures)

Async IO is built into `asyncio` library, which is to write concurrent code using the `async/await` syntax.

In [4]:
import asyncio

### Intro to `asyncio` functions

- `asyncio.sleep(delay, result=None)` is an awaitable, meaning it can be used with `await` to pause the coroutine. Its a coroutine that suspends the current task, allowing other tasks to run. While `time.sleep()` can represent any time-consuming blocking function call, `asyncio.sleep()` is used to stand in for a non-blocking call (but one that also takes some time to complete).
- `asyncio.run(coro, *, debug=None, loop_factory=None)` is a function that executes the coroutine `coro` and returns the result.

## Coroutines

**Coroutines** are the foundation of Async IO's cooperative multitasking model. They're special functions that can pause execution, yield control back to the event loop and later resume from where they left off.

1. **Definition**: Native coroutines are defined using the `async def` syntax
2. **Suspension Points**: They can suspend execution at `await` expressions
3. **State Preservation**: When suspended, they maintain their state (local variables, execution position)
4. **Non-blocking**: They don't block the thread while waiting for operations to complete

In [5]:
async def say_hello():
    print("Hello")
    await asyncio.sleep(1)
    print("World")

In [6]:
say_hello

<function __main__.say_hello()>

In [7]:
say_hello()

<coroutine object say_hello at 0x1071f0e80>

**See practical example 1.**

### More `asyncio` functions

- #### `asyncio.gather(*aws, return_exceptions=False)`

Run awaitable objects in the `aws` sequence concurrently. It is used to run multiple coroutines at the same time, and wait until all of them complete.

If `return_exceptions` is `False` (default), the first raised exception is immediately propagated to the task that awaits on `gather()`. Other awaitables in the `aws` sequence won’t be cancelled and will continue to run.

If `return_exceptions` is `True`, exceptions are treated the same as successful results, and aggregated in the result list.

**See practical example 2.**

**See practical example 3.**

## Tasks

**Tasks** are one of the core abstractions in Async IO that build upon coroutines. A `Task` is a wrapper around a coroutine that schedules it for execution in the event loop, tracks its state and provides additional functionality for managing its execution.

1. **Definition**: A `Task` is a `Future-like` object that runs a coroutine in an event loop
2. **Purpose**: Tasks allow coroutines to execute concurrently within a single thread
3. **Scheduling**: They're scheduled automatically by the event loop once created
4. **Status Tracking**: They track the completion status of the wrapped coroutine

- #### `asyncio.create_task(coro, *, name=None, context=None)`

This function wraps the `coro` coroutine into a `Task` and schedule its execution. It returns the `Task` object. It is used to turn a coroutine into a `Task` and schedule it to run in the background on the event loop right away. It's essential when you want to start an async operation without waiting for it to finish immediately.

- #### `asyncio.wait_for(awaitable, timeout)`

This waits for the `aw` awaitable to complete with a `timeout`. If a timeout occurs, it cancels the task and raises `TimeoutError`.

**See practical example 4.**

**See practical example 5.**

## Event loop

The **event loop** is the central execution mechanism in Async IO that orchestrates running asynchronous tasks. It's responsible for scheduling coroutines, handling I/O operations and managing the flow of control in your asynchronous application.

1. **Definition**: The event loop is a programming construct that waits for and dispatches events or messages in a program
2. **Role**: It manages all the asynchronous operations, running coroutines one by one when they're ready to execute
3. **Single-threaded**: It runs on a single thread, executing tasks sequentially but allowing them to cooperatively yield control

Usually we run an Async IO program as follows (recommended way), which implicitly starts the event loop.

```python
import asyncio


async def main():
    print("Hello")
    await asyncio.sleep(1)
    print("World")


if __name__ == "__main__":
    asyncio.run(main())
```

- #### `asyncio.new_event_loop()`

Create and return a new event loop object.

- #### `asyncio.get_running_loop()`

Return the running event loop in the current OS thread.

Raise a `RuntimeError` if there is no running event loop.

This function can only be called from a coroutine or a callback.

- #### `loop.run_until_complete(future)`

Run until the `future` (an instance of `Future`) has completed.

If the argument is a coroutine object it is implicitly scheduled to run as a `asyncio.Task`.

Return the Future’s result or raise its exception.

- #### `loop.close()`

Close the event loop.

The loop must not be running when this function is called. Any pending callbacks will be discarded.

This method clears all queues and shuts down the executor, but does not wait for the executor to finish.

This method is idempotent and irreversible. No other methods should be called after the event loop is closed.

### Key points

#### 1. Coroutines don’t do much on their own until they are tied to the event loop.

Remember to use `asyncio.run()` to actually force execution by scheduling the `main()` coroutine (future object) for execution on the event loop.

In [8]:
async def main():
    print("Hello ...")
    await asyncio.sleep(1)
    print("World!")
    
main()

<coroutine object main at 0x1071f1f00>

**See practical example 6.**

#### 2. By default, an async IO event loop runs in a single thread and on a single CPU core.

Usually, running one single-threaded event loop in one CPU core is more than sufficient. It is also possible to run event loops across multiple cores.

#### 3. Event loops are pluggable. 

That is, you could, if you really wanted, write your own event loop implementation and have it run tasks just the same. For example, [uvloop](https://github.com/MagicStack/uvloop) is a fast event loop implementation.

## Async iterators and generators

The `async for` statement allows you to iterate over an asynchronous iterator, pausing execution at each iteration to wait for the next item without blocking the event loop. This is particularly useful when working with streaming data, asynchronous data sources, or when processing large datasets incrementally.

### Key concepts

1. **Asynchronous Iterators**: Objects that implement the `__aiter__()` and `__anext__()` methods.
2. **Asynchronous Comprehensions**: Similar to regular comprehensions but using `async for`.
3. **Asynchronous Generators**: Functions that use `yield` inside `async def` functions.

**See practical example 7 about *async iterators*.**

**See practical example 8 about *async generators and comprehensions*.**

## Async context manager

The `async with` statement is designed to work with asynchronous context managers, allowing you to use resources that require setup and cleanup operations in an asynchronous environment. It's similar to the standard `with` statement but works with `async`/`await` syntax.

### Key Concepts

1. **Asynchronous Context Managers**: Objects that implement the `__aenter__()` and `__aexit__()` methods
2. **Resource Management**: Automatically handles acquisition and release of resources
3. **Error Handling**: Manages exceptions that occur within the context block

**See practical example 9.**

## More asyncio functions

- #### `asyncio.as_completed(aws, *, timeout=None)`

Run awaitable objects in the `aws` iterable concurrently. The returned object can be iterated to obtain the results of the awaitables as they finish.

The object returned by `as_completed()` can be iterated as an asynchronous iterator or a plain iterator.

- #### `async asyncio.to_thread(func, /, *args, **kwargs)`

Asynchronously run function `func` in a separate thread.

Any `*args` and `**kwargs` supplied for this function are directly passed to `func`.

Return a coroutine that can be awaited to get the eventual result of `func`.

This coroutine function is primarily intended to be used for executing IO-bound functions/methods that would otherwise block the event loop if they were run in the main thread.

**See practical example 10.**

## Synchronization

There are the usual synchronization primitives in `asyncio` as follows:

- `Lock`
- `Event`
- `Semaphore`
- etc.

### `asyncio.Semaphore(value=1)`

A `Semaphore` object. Not thread-safe.

A semaphore manages an internal counter which is decremented by each `acquire()` call and incremented by each `release()` call. The counter can never go below zero; when `acquire()` finds that it is zero, it blocks, waiting until some task calls `release()`.

The optional value argument gives the initial value for the internal counter (1 by default). If the given value is less than 0 a `ValueError` is raised.

Usually, a semaphore can be used to limit the number of concurrent accessors to a shared resource — like a database, file, or network call.

**See practical example 11.**

## Web scraper example

**See practical example 12.**